<a href="https://colab.research.google.com/github/keripikkaneboo/Hands-On-Machine-Learning-O-Reilly-/blob/main/04.%20Chapter4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Bab 4: Training Models

Bab ini menyelam lebih dalam ke algoritma-algoritma Machine Learning, dengan fokus pada **Model Linier**. Bab ini menjelaskan dua cara yang sangat berbeda untuk melatihnya:
1.  Menggunakan solusi matematis langsung (*closed-form equation*).
2.  Menggunakan pendekatan optimisasi iteratif (*Gradient Descent*).

Selain itu, bab ini juga mencakup regresi polinomial, cara mendiagnosis *overfitting* dan *underfitting* dengan *learning curves*, dan model yang umum digunakan untuk klasifikasi seperti *Logistic Regression* dan *Softmax Regression*.

* **Regresi Linier**: Model paling sederhana yang membuat prediksi dengan menghitung jumlah terbobot dari fitur-fitur input, ditambah *bias term*.
    * ***Normal Equation***: Sebuah persamaan matematis yang memberikan hasil parameter optimal secara langsung. Sangat efisien untuk dataset yang tidak terlalu besar dan jumlah fitur yang tidak terlalu banyak.
    * ***Gradient Descent (GD)***: Algoritma optimisasi generik yang secara bertahap menyesuaikan parameter untuk meminimalkan *cost function*.
        * ***Batch GD***: Menggunakan seluruh set data training di setiap langkahnya.
        * ***Stochastic GD (SGD)***: Menggunakan hanya satu instance acak di setiap langkah, membuatnya jauh lebih cepat tetapi kurang stabil.
        * ***Mini-batch GD***: Kompromi antara keduanya, menggunakan sekelompok kecil instance acak (*mini-batch*) di setiap langkah.

* **Regresi Polinomial**: Teknik untuk menggunakan model linier pada data non-linier dengan cara menambahkan pangkat dari fitur sebagai fitur baru.

* ***Learning Curves***: Alat untuk mendiagnosis apakah sebuah model mengalami *overfitting* atau *underfitting*. Kurva ini memplot performa model pada *training set* dan *validation set* sebagai fungsi dari ukuran *training set*.

* **Model Linier yang Diregularisasi**: Teknik untuk mengurangi *overfitting* dengan membatasi bobot (*weights*) model.
    * ***Ridge Regression***: Menambahkan penalti ℓ₂ pada *cost function*.
    * ***Lasso Regression***: Menambahkan penalti ℓ₁ yang cenderung membuat bobot fitur yang tidak penting menjadi nol (melakukan seleksi fitur otomatis).
    * ***Elastic Net***: Kombinasi dari Ridge dan Lasso.
    * ***Early Stopping***: Bentuk regularisasi dengan menghentikan training ketika error pada *validation set* berhenti menurun.

* **Regresi Logistik dan Softmax**:
    * ***Logistic Regression***: Digunakan untuk tugas klasifikasi biner dengan mengestimasi probabilitas.
    * ***Softmax Regression***: Generalisasi dari Regresi Logistik untuk mendukung klasifikasi multikelas.

### 1. Regresi Linier dengan Normal Equation
Kita bisa menghitung parameter ($\theta$) optimal secara langsung.

```python
import numpy as np
import matplotlib.pyplot as plt

# Membuat data linier acak
X = 2 * np.random.rand(100, 1)
y = 4 + 3 * X + np.random.randn(100, 1)

# Menambahkan x0 = 1 ke setiap instance (untuk bias term)
X_b = np.c_[np.ones((100, 1)), X]

# Menghitung theta terbaik menggunakan Normal Equation
theta_best = np.linalg.inv(X_b.T.dot(X_b)).dot(X_b.T).dot(y)

print("Theta terbaik yang dihitung dengan Normal Equation:")
print(theta_best)

# Membuat prediksi menggunakan theta yang ditemukan
X_new = np.array([[0], [2]])
X_new_b = np.c_[np.ones((2, 1)), X_new]
y_predict = X_new_b.dot(theta_best)

print("\nPrediksi untuk X_new:")
print(y_predict)

# Plot data dan garis regresi
plt.plot(X, y, "b.")
plt.plot(X_new, y_predict, "r-", label="Prediksi")
plt.xlabel("$x_1$")
plt.ylabel("$y$")
plt.axis([0, 2, 0, 15])
plt.legend()
plt.show()

# Melakukan hal yang sama dengan Scikit-Learn
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(X, y)
print("\nScikit-Learn intercept dan coefficient:")
print(lin_reg.intercept_, lin_reg.coef_)
```

### 2. Regresi Linier dengan Batch Gradient Descent
Implementasi manual untuk memahami cara kerja algoritma optimisasi iteratif.

```python
eta = 0.1  # learning rate
n_iterations = 1000
m = 100 # jumlah instance

theta = np.random.randn(2, 1)  # inisialisasi acak

for iteration in range(n_iterations):
    gradients = 2/m * X_b.T.dot(X_b.dot(theta) - y)
    theta = theta - eta * gradients

print("Theta terbaik yang dihitung dengan Batch Gradient Descent:")
print(theta)
```

### 3. Regresi Polinomial
Menggunakan model linier untuk data non-linier.

```python
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

# Membuat data non-linier
m = 100
X = 6 * np.random.rand(m, 1) - 3
y = 0.5 * X**2 + X + 2 + np.random.randn(m, 1)

# Menambahkan fitur polinomial (derajat 2)
poly_features = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly_features.fit_transform(X)

# Melatih model Linear Regression pada data yang sudah ditransformasi
lin_reg = LinearRegression()
lin_reg.fit(X_poly, y)

print("Intercept dan Coefficient dari Regresi Polinomial:")
print(lin_reg.intercept_, lin_reg.coef_)

# Plot hasil
X_new=np.linspace(-3, 3, 100).reshape(100, 1)
X_new_poly = poly_features.transform(X_new)
y_new = lin_reg.predict(X_new_poly)
plt.plot(X, y, "b.")
plt.plot(X_new, y_new, "r-", linewidth=2, label="Prediksi")
plt.xlabel("$x_1$")
plt.ylabel("$y$")
plt.legend(loc="upper left")
plt.axis([-3, 3, 0, 10])
plt.show()
```

### 4. Model yang Diregularisasi
Contoh penggunaan Ridge, Lasso, dan Elastic Net untuk mencegah *overfitting*.

```python
from sklearn.linear_model import Ridge, Lasso, ElasticNet

# Menggunakan data yang sama dengan Regresi Polinomial
# Misalkan X_poly dan y sudah ada

# Ridge Regression
ridge_reg = Ridge(alpha=1, solver="cholesky")
ridge_reg.fit(X_poly, y)
print("Ridge Intercept & Coef:", ridge_reg.intercept_, ridge_reg.coef_)

# Lasso Regression
lasso_reg = Lasso(alpha=0.1)
lasso_reg.fit(X_poly, y)
print("Lasso Intercept & Coef:", lasso_reg.intercept_, lasso_reg.coef_)

# Elastic Net
elastic_net = ElasticNet(alpha=0.1, l1_ratio=0.5)
elastic_net.fit(X_poly, y)
print("ElasticNet Intercept & Coef:", elastic_net.intercept_, elastic_net.coef_)
```

### 5. Regresi Logistik
Contoh klasifikasi biner menggunakan dataset Iris.

```python
from sklearn import datasets
from sklearn.linear_model import LogisticRegression

# Memuat dataset Iris
iris = datasets.load_iris()
X = iris["data"][:, 3:]  # fitur petal width
y = (iris["target"] == 2).astype(int)  # 1 jika Iris-Virginica, else 0

# Melatih model Logistic Regression
log_reg = LogisticRegression()
log_reg.fit(X, y)

# Membuat prediksi probabilitas
X_new = np.linspace(0, 3, 1000).reshape(-1, 1)
y_proba = log_reg.predict_proba(X_new)

# Plot hasil
plt.plot(X_new, y_proba[:, 1], "g-", label="Iris-Virginica")
plt.plot(X_new, y_proba[:, 0], "b--", label="Bukan Iris-Virginica")
plt.xlabel("Petal width (cm)")
plt.ylabel("Probabilitas")
plt.legend()
plt.show()

# Prediksi kelas
print("\nPrediksi untuk petal width 1.7cm:", log_reg.predict([[1.7]]))
print("Prediksi untuk petal width 1.5cm:", log_reg.predict([[1.5]]))
```
